<a href="https://colab.research.google.com/github/Mirthu24/-Guvi-Project_1/blob/main/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries
import selenium as st
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException,ElementClickInterceptedException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np

# 1.Kerala

In [ ]:

# Open the browser
driver_1 = webdriver.Chrome()

# Load the webpage
driver_1.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")

time.sleep(2)  # Wait for the page to load

driver_1.maximize_window()

# Retrieve bus links and routes
wait = WebDriverWait(driver_1, 30)

def Kerala_link_route(path):
    Kerala_link = []
    Kerala_route = []

    # Retrieve the route links
    for i in range(1, 3):
        paths = driver_1.find_elements(By.XPATH, path)

        for link in paths:
            d = link.get_attribute("href")
            Kerala_link.append(d)

        # Retrieve names of the routes
        for route in paths:
            Kerala_route.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_button = pagination.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()={i+1}]')
            time.sleep(2)

            # Retry clicking if intercepted
            retries = 3
            while retries > 0:
                try:
                    next_button.click()
                    break
                except ElementClickInterceptedException:
                    time.sleep(1)
                    retries -= 1
                    if retries == 0:
                        raise

        except NoSuchElementException:
            print(f"Total number of pages found {i}")
            break

    return Kerala_link, Kerala_route

Kerala_link, Kerala_route = Kerala_link_route("//a[@class='route']")

Total number of pages found 2


In [ ]:
df_k=pd.DataFrame({"Route_name":Kerala_route,"Route_link":Kerala_link})
df_k

,Route_name,Route_link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Bangalore to Kannur,https://www.redbus.in/bus-tickets/bangalore-to...
5,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
6,Kannur to Bangalore,https://www.redbus.in/bus-tickets/kannur-to-ba...
7,Kozhikode to Thiruvananthapuram,https://www.redbus.in/bus-tickets/kozhikode-to...
8,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
9,Bangalore to Kalpetta (kerala),https://www.redbus.in/bus-tickets/bangalore-to...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Kerala.csv"
df_k.to_csv(path,index=False)

In [ ]:
# Define lists to store the bus details

def  scroll():
    x=0
    l=[]
    while True:
        x=x+1
        driver_1.execute_script('scrollBy(0,100)')
        time.sleep(0.5)
        if(x>1000):
            break

wait = WebDriverWait(driver_1,20)

Kerala_bus_data=[]

def scrape_data(Kerala_link):

    # Open the route link
    driver_1.get(Kerala_link)
    time.sleep(3)
    Busroute=driver_1.find_element(By.XPATH,"//h1[@class='D136_h1']").text
    # Click to view bus details
    try:
        b=driver_1.find_elements(By.CSS_SELECTOR,"div[class='button']")
        for i in range(len(b)-1,-1,-1):
            b[i].click()

        # Scroll to the bottom of the page
        scroll()

        bus_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
        for bus in bus_items:
            try:
                busname = bus.find_element(By.CLASS_NAME, "travels").text
                bustype = bus.find_element(By.CLASS_NAME, "bus-type").text
                departing_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                duration = bus.find_element(By.CLASS_NAME, "dur").text
                reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text
                star_rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                price = bus.find_element(By.CSS_SELECTOR, ".seat-fare .fare span").text
                seats_available = bus.find_element(By.CLASS_NAME, "seat-left").text

                Kerala_bus_data.append({
                    "Busname": busname,
                    "Bustype": bustype,
                    "Busroutes": Busroute,
                    "Departing_time": departing_time,
                    "Duration": duration,
                    "Reaching_time": reaching_time,
                    "Star_rating": star_rating,
                    "Price": price,
                    "Seats_available": seats_available,
                    "buslink": Kerala_link
                })

            except Exception as e:
                print(f"An error occurred while scraping the data: {e}")
    except Exception as e:
        print(f"An error occurred while trying to click the view buttons: {e}")

print("Data loaded successfully")

for i in Kerala_link:
    scrape_data(i)
    time.sleep(3)

Data loaded successfully
An error occurred while scraping the data: Message: no such element: Unable to locate element: {"method":"css selector","selector":".rating-sec .rating span"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7FA87B125+29573]
	(No symbol) [0x00007FF7FA7EFF50]
	(No symbol) [0x00007FF7FA6AB6EA]
	(No symbol) [0x00007FF7FA6FF815]
	(No symbol) [0x00007FF7FA6FFA6C]
	(No symbol) [0x00007FF7FA6F312C]
	(No symbol) [0x00007FF7FA72733F]
	(No symbol) [0x00007FF7FA6F2FF6]
	(No symbol) [0x00007FF7FA727510]
	(No symbol) [0x00007FF7FA7486BC]
	(No symbol) [0x00007FF7FA7270A3]
	(No symbol) [0x00007FF7FA6F12DF]
	(No symbol) [0x00007FF7FA6F2441]
	GetHandleVerifier [0x00007FF7FABAC76D+3377613]
	GetHandleVerifier [0x00007FF7FABF7B67+3685831]
	GetHandleVerifier [0x00007FF7FABECF8B+3641835]
	GetHandleVerifier [0

In [ ]:
df_buses_Kerala = pd.DataFrame(Kerala_bus_data)
df_buses_Kerala

,Busname,Bustype,Busroutes,Departing_time,Duration,Reaching_time,Star_rating,Price,Seats_available,buslink
0,KSRTC (Kerala) - 581,Super Express Non AC Seater Air Bus (2+2),Bangalore to Kozhikode Bus,10:00,08h 22m,18:22,4.1,469,7 Seats available,https://www.redbus.in/bus-tickets/bangalore-to...
1,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,Bangalore to Kozhikode Bus,12:00,08h 20m,20:20,3.7,627,8 Seats available,https://www.redbus.in/bus-tickets/bangalore-to...
2,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,Bangalore to Kozhikode Bus,14:15,08h 50m,23:05,4.3,627,32 Seats available,https://www.redbus.in/bus-tickets/bangalore-to...
3,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),Bangalore to Kozhikode Bus,15:00,10h 00m,01:00,3.9,620,29 Seats available,https://www.redbus.in/bus-tickets/bangalore-to...
4,KSRTC (Kerala) - 370,AC MULTI AXLE,Bangalore to Kozhikode Bus,15:25,06h 55m,22:20,2.7,729,5 Seats available,https://www.redbus.in/bus-tickets/bangalore-to...
...,...,...,...,...,...,...,...,...,...,...
469,SKS Tourist Corporation (REGD),A/C Sleeper (2+1),Mysore to Kozhikode Bus,23:55,05h 35m,05:30,3.5,899,21 Seats available,https://www.redbus.in/bus-tickets/mysore-to-ko...
470,SKS Tours & Travels,A/C Sleeper (2+1),Mysore to Kozhikode Bus,23:59,05h 41m,05:40,3.6,899,25 Seats available,https://www.redbus.in/bus-tickets/mysore-to-ko...
471,NS Transports,VE A/C Sleeper (2+1),Mysore to Kozhikode Bus,00:45,06h 45m,07:30,4.1,924,21 Seats available,https://www.redbus.in/bus-tickets/mysore-to-ko...
472,Kallada Travels (Suresh Kallada),A/C Sleeper (2+1),Mysore to Kozhikode Bus,05:20,05h 40m,11:00,2.5,1600,23 Seats available,https://www.redbus.in/bus-tickets/mysore-to-ko...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Bus_Kerala.csv"
df_buses_Kerala.to_csv(path,index=False)

# 2.Andhra Pradesh

In [ ]:

# Open the browser
driver_2 = webdriver.Chrome()

# Load the webpage
driver_2.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")

time.sleep(2)  # Wait for the page to load

driver_2.maximize_window()

# Retrieve bus links and routes
wait = WebDriverWait(driver_2, 30)

def Andhra_link_route(path):
    Andhra_link = []
    Andhra_route = []

    # Retrieve the route links
    for i in range(1, 6):
        paths = driver_2.find_elements(By.XPATH, path)

        for link in paths:
            d = link.get_attribute("href")
            Andhra_link.append(d)

        # Retrieve names of the routes
        for route in paths:
            Andhra_route.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_button = pagination.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()={i+1}]')
            time.sleep(2)

            # Retry clicking if intercepted
            retries = 6
            while retries > 0:
                try:
                    next_button.click()
                    break
                except ElementClickInterceptedException:
                    time.sleep(1)
                    retries -= 1
                    if retries == 0:
                        raise

        except NoSuchElementException:
            print(f"Total number of pages found {i}")
            break

    return Andhra_link, Andhra_route

Andhra_link, Andhra_route = Andhra_link_route("//a[@class='route']")


Total number of pages found 5


In [ ]:
df_AP=pd.DataFrame({"Route_name":Andhra_route,"Route_link":Andhra_link})
df_AP


,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
2,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
4,Bangalore to Tirupati,https://www.redbus.in/bus-tickets/bangalore-to...
5,Bangalore to Kadapa,https://www.redbus.in/bus-tickets/bangalore-to...
6,Ongole to Hyderabad,https://www.redbus.in/bus-tickets/ongole-to-hy...
7,Kadapa to Bangalore,https://www.redbus.in/bus-tickets/kadapa-to-ba...
8,Chittoor (Andhra Pradesh) to Bangalore,https://www.redbus.in/bus-tickets/chittoor-and...
9,Visakhapatnam to Kakinada,https://www.redbus.in/bus-tickets/visakhapatna...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Andhra.csv"
df_AP.to_csv(path,index=False)

In [ ]:
# Define lists to store the bus details

def  scroll():
    x=0
    l=[]
    while True:
        x=x+1
        driver_2.execute_script('scrollBy(0,100)')
        time.sleep(0.5)
        if(x>1000):
            break

wait = WebDriverWait(driver_2,20)

Andhra_bus_data=[]

def scrape_data(Andhra_link):

    # Open the route link
    driver_2.get(Andhra_link)
    time.sleep(3)

    Busroute=driver_2.find_element(By.XPATH,"//h1[@class='D136_h1']").text

    # Click to view bus details
    try:
        b=driver_2.find_elements(By.CSS_SELECTOR,"div[class='button']")
        for i in range(len(b)-1,-1,-1):
            b[i].click()

        # Scroll to the bottom of the page
        scroll()

        bus_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
        for bus in bus_items:
            try:
                busname = bus.find_element(By.CLASS_NAME, "travels").text
                bustype = bus.find_element(By.CLASS_NAME, "bus-type").text
                departing_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                duration = bus.find_element(By.CLASS_NAME, "dur").text
                reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text
                star_rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                price = bus.find_element(By.CSS_SELECTOR, ".seat-fare .fare span").text
                seats_available = bus.find_element(By.CLASS_NAME, "seat-left").text

                Andhra_bus_data.append({
                    "Busname": busname,
                    "Bustype": bustype,
                    "Busroutes": Busroute,
                    "Departing_time": departing_time,
                    "Duration": duration,
                    "Reaching_time": reaching_time,
                    "Star_rating": star_rating,
                    "Price": price,
                    "Seats_available": seats_available,
                    "buslink": Andhra_link
                })

            except Exception as e:
                print(f"An error occurred while scraping the data: {e}")
    except Exception as e:
        print(f"An error occurred while trying to click the view buttons: {e}")

print("Data loaded successfully")

for i in Andhra_link:
    scrape_data(i)
    time.sleep(3)

Data loaded successfully
An error occurred while scraping the data: Message: no such element: Unable to locate element: {"method":"css selector","selector":".rating-sec .rating span"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF728B0B125+29573]
	(No symbol) [0x00007FF728A7FF50]
	(No symbol) [0x00007FF72893B6EA]
	(No symbol) [0x00007FF72898F815]
	(No symbol) [0x00007FF72898FA6C]
	(No symbol) [0x00007FF72898312C]
	(No symbol) [0x00007FF7289B733F]
	(No symbol) [0x00007FF728982FF6]
	(No symbol) [0x00007FF7289B7510]
	(No symbol) [0x00007FF7289D86BC]
	(No symbol) [0x00007FF7289B70A3]
	(No symbol) [0x00007FF7289812DF]
	(No symbol) [0x00007FF728982441]
	GetHandleVerifier [0x00007FF728E3C76D+3377613]
	GetHandleVerifier [0x00007FF728E87B67+3685831]
	GetHandleVerifier [0x00007FF728E7CF8B+3641835]
	GetHandleVerifier [0

In [ ]:
df_buses_Andhra = pd.DataFrame(Andhra_bus_data)
df_buses_Andhra

,Busname,Bustype,Busroutes,Departing_time,Duration,Reaching_time,Star_rating,Price,Seats_available,buslink
0,FRESHBUS,Electric A/C Seater (2+2),Hyderabad to Vijayawada Bus,06:25,05h 20m,11:45,4.7,450,8 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
1,FRESHBUS,Electric A/C Seater (2+2),Hyderabad to Vijayawada Bus,23:10,06h 25m,05:35,4.6,480,31 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
2,FRESHBUS,Electric A/C Seater (2+2),Hyderabad to Vijayawada Bus,11:05,06h 35m,17:40,4.6,500,33 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
3,AdIntrCity SmartBus,A/C Seater / Sleeper (2+1),Hyderabad to Vijayawada Bus,00:15,06h 10m,06:25,4.1,425,38 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
4,IntrCity SmartBus,A/C Seater / Sleeper (2+1),Hyderabad to Vijayawada Bus,23:50,05h 45m,05:35,4.4,523,35 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
...,...,...,...,...,...,...,...,...,...,...
4641,Deepna Travels,Non A/C Seater / Sleeper (2+1),Hyderabad to Nandyal Bus,22:00,06h 00m,04:00,3.6,600,23 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
4642,AR & BCVR Travels,Non A/C Seater / Sleeper (2+1),Hyderabad to Nandyal Bus,22:15,05h 15m,03:30,3.5,700,17 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
4643,Matangi Travels,Non A/C Seater / Sleeper (2+1),Hyderabad to Nandyal Bus,22:10,06h 15m,04:25,3.4,523,18 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
4644,True Bus,Volvo Multi Axle B9R A/C Sleeper (2+1),Hyderabad to Nandyal Bus,22:30,06h 00m,04:30,3.2,1180,30 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Bus_Andhra.csv"
df_buses_Andhra.to_csv(path,index=False)

# 3.Telangana

In [ ]:
# Open the browser
driver_3 = webdriver.Chrome()

# Load the webpage
driver_3.get("https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile")

time.sleep(2)  # Wait for the page to load

driver_3.maximize_window()

# Retrieve bus links and routes
wait = WebDriverWait(driver_3, 10)

def Telangana_link_route(path):
    Telangana_link = []
    Telangana_route = []

    # Retrieve the route links
    for i in range(1, 4):
        paths = driver_3.find_elements(By.XPATH, path)

        for link in paths:
            d = link.get_attribute("href")
            Telangana_link.append(d)

        # Retrieve names of the routes
        for route in paths:
            Telangana_route.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_button = pagination.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()={i+1}]')
            time.sleep(2)

            # Retry clicking if intercepted
            retries = 4
            while retries > 0:
                try:
                    next_button.click()
                    break
                except ElementClickInterceptedException:
                    time.sleep(1)
                    retries -= 1
                    if retries == 0:
                        raise

        except NoSuchElementException:
            print(f"Total number of pages found {i}")
            break

    return Telangana_link, Telangana_route

Telangana_link, Telangana_route = Telangana_link_route("//a[@class='route']")

Total number of pages found 3


In [ ]:
df_TS=pd.DataFrame({"Route_name":Telangana_route,"Route_link":Telangana_link})
df_TS

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
2,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
5,Hyderabad to Karimnagar,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
7,Hyderabad to Nirmal,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
9,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Telangana.csv"
df_TS.to_csv(path,index=False)

In [ ]:
# Define lists to store the bus details

def  scroll():
    x=0
    l=[]
    while True:
        x=x+1
        driver_3.execute_script('scrollBy(0,100)')
        time.sleep(0.5)
        if(x>1000):
            break

wait = WebDriverWait(driver_3,20)

Telangana_bus_data=[]

def scrape_data(Telangana_link):

    # Open the route link
    driver_3.get(Telangana_link)
    time.sleep(3)
    Busroute=driver_3.find_element(By.XPATH,"//h1[@class='D136_h1']").text
    # Click to view bus details
    try:
        b=driver_3.find_elements(By.CSS_SELECTOR,"div[class='button']")
        for i in range(len(b)-1,-1,-1):
            b[i].click()

        # Scroll to the bottom of the page
        scroll()

        bus_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
        for bus in bus_items:
            try:
                busname = bus.find_element(By.CLASS_NAME, "travels").text
                bustype = bus.find_element(By.CLASS_NAME, "bus-type").text
                departing_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                duration = bus.find_element(By.CLASS_NAME, "dur").text
                reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text
                star_rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                price = bus.find_element(By.CSS_SELECTOR, ".seat-fare .fare span").text
                seats_available = bus.find_element(By.CLASS_NAME, "seat-left").text

                Telangana_bus_data.append({
                    "Busname": busname,
                    "Bustype": bustype,
                    "Busroutes": Busroute,
                    "Departing_time": departing_time,
                    "Duration": duration,
                    "Reaching_time": reaching_time,
                    "Star_rating": star_rating,
                    "Price": price,
                    "Seats_available": seats_available,
                    "buslink": Telangana_link
                })

            except Exception as e:
                print(f"An error occurred while scraping the data: {e}")
    except Exception as e:
        print(f"An error occurred while trying to click the view buttons: {e}")

for i in Telangana_link:
    scrape_data(i)
    time.sleep(3)

print("Data loaded successfully")

An error occurred while scraping the data: Message: no such element: Unable to locate element: {"method":"css selector","selector":".rating-sec .rating span"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF79D1CB645+29573]
	(No symbol) [0x00007FF79D140470]
	(No symbol) [0x00007FF79CFFB6EA]
	(No symbol) [0x00007FF79D04F815]
	(No symbol) [0x00007FF79D04FA6C]
	(No symbol) [0x00007FF79D04312C]
	(No symbol) [0x00007FF79D07733F]
	(No symbol) [0x00007FF79D042FF6]
	(No symbol) [0x00007FF79D077510]
	(No symbol) [0x00007FF79D0986BC]
	(No symbol) [0x00007FF79D0770A3]
	(No symbol) [0x00007FF79D0412DF]
	(No symbol) [0x00007FF79D042441]
	GetHandleVerifier [0x00007FF79D4FC58D+3375821]
	GetHandleVerifier [0x00007FF79D547987+3684039]
	GetHandleVerifier [0x00007FF79D53CDAB+3640043]
	GetHandleVerifier [0x00007FF79D28B7C6+816390]

In [ ]:
Telangana = pd.DataFrame(Telangana_bus_data)
Telangana

,Busname,Bustype,Busroutes,Departing_time,Duration,Reaching_time,Star_rating,Price,Seats_available,buslink
0,APSRTC - 3376,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",Hyderabad to Vijayawada Bus,16:45,06h 30m,23:15,4.2,436,23 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
1,APSRTC - 2561,INDRA(A.C. Seater),Hyderabad to Vijayawada Bus,17:15,05h 30m,22:45,3.1,528,10 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
2,APSRTC - 3358,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",Hyderabad to Vijayawada Bus,17:20,07h 50m,01:10,2.5,469,20 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
3,APSRTC - 2674,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",Hyderabad to Vijayawada Bus,17:30,07h 50m,01:20,2.6,436,19 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
4,APSRTC - 3366,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",Hyderabad to Vijayawada Bus,17:55,07h 50m,01:45,2.4,469,33 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
...,...,...,...,...,...,...,...,...,...,...
1104,KTC Travels,NON A/C Sleeper (2+1),Hyderabad to Addanki Bus,21:25,06h 35m,04:00,2.9,1499,6 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
1105,VIHARI TRAVELS,NON A/C Sleeper (2+1),Hyderabad to Addanki Bus,22:00,07h 00m,05:00,2.5,1999,2 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
1106,VIHARI TRAVELS,NON A/C Sleeper (2+1),Hyderabad to Addanki Bus,22:00,07h 45m,05:45,2.5,1999,1 Seat available,https://www.redbus.in/bus-tickets/hyderabad-to...
1107,Sai Durga Travels,Non A/C Seater / Sleeper (2+1),Hyderabad to Addanki Bus,22:40,08h 20m,07:00,1.4,5000,1 Seat available,https://www.redbus.in/bus-tickets/hyderabad-to...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Bus_Telangana_d.csv"
Telangana.to_csv(path,index=False)

In [ ]:
bus_Telangana = pd.read_csv("Bus_Telangana_d.csv")

In [ ]:
df_buses_Telangana = bus_Telangana.dropna()
df_buses_Telangana

,Busname,Bustype,Busroutes,Departing_time,Duration,Reaching_time,Star_rating,Price,Seats_available,buslink
0,APSRTC - 3376,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",Hyderabad to Vijayawada Bus,16:45,06h 30m,23:15,4.2,436.0,23 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
1,APSRTC - 2561,INDRA(A.C. Seater),Hyderabad to Vijayawada Bus,17:15,05h 30m,22:45,3.1,528.0,10 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
2,APSRTC - 3358,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",Hyderabad to Vijayawada Bus,17:20,07h 50m,01:10,2.5,469.0,20 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
3,APSRTC - 2674,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",Hyderabad to Vijayawada Bus,17:30,07h 50m,01:20,2.6,436.0,19 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
4,APSRTC - 3366,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",Hyderabad to Vijayawada Bus,17:55,07h 50m,01:45,2.4,469.0,33 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
...,...,...,...,...,...,...,...,...,...,...
1104,KTC Travels,NON A/C Sleeper (2+1),Hyderabad to Addanki Bus,21:25,06h 35m,04:00,2.9,1499.0,6 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
1105,VIHARI TRAVELS,NON A/C Sleeper (2+1),Hyderabad to Addanki Bus,22:00,07h 00m,05:00,2.5,1999.0,2 Seats available,https://www.redbus.in/bus-tickets/hyderabad-to...
1106,VIHARI TRAVELS,NON A/C Sleeper (2+1),Hyderabad to Addanki Bus,22:00,07h 45m,05:45,2.5,1999.0,1 Seat available,https://www.redbus.in/bus-tickets/hyderabad-to...
1107,Sai Durga Travels,Non A/C Seater / Sleeper (2+1),Hyderabad to Addanki Bus,22:40,08h 20m,07:00,1.4,5000.0,1 Seat available,https://www.redbus.in/bus-tickets/hyderabad-to...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Bus_Telangana.csv"
df_buses_Telangana.to_csv(path,index=False)

# 4.Rajasthan

In [ ]:

# Open the browser
driver_4 = webdriver.Chrome()

# Load the webpage
driver_4.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile")

time.sleep(2)  # Wait for the page to load

driver_4.maximize_window()

# Retrieve bus links and routes
wait = WebDriverWait(driver_4, 10)

def Rajasthan_link_route(path):
    Rajasthan_link = []
    Rajasthan_route = []

    # Retrieve the route links
    for i in range(1, 3):
        paths = driver_4.find_elements(By.XPATH, path)

        for link in paths:
            d = link.get_attribute("href")
            Rajasthan_link.append(d)

        # Retrieve names of the routes
        for route in paths:
            Rajasthan_route.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_button = pagination.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()={i+1}]')
            time.sleep(2)

            # Retry clicking if intercepted
            retries = 3
            while retries > 0:
                try:
                    next_button.click()
                    break
                except ElementClickInterceptedException:
                    time.sleep(1)
                    retries -= 1
                    if retries == 0:
                        raise

        except NoSuchElementException:
            print(f"Total number of pages found {i}")
            break

    return Rajasthan_link, Rajasthan_route

Rajasthan_link, Rajasthan_route = Rajasthan_link_route("//a[@class='route']")

Total number of pages found 2


In [ ]:
df_RJ=pd.DataFrame({"Route_name":Rajasthan_route,"Route_link":Rajasthan_link})
df_RJ

,Route_name,Route_link
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
2,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
3,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
4,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
5,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
6,Jaipur (Rajasthan) to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/jaipur-to-al...
7,Jodhpur to Beawar (Rajasthan),https://www.redbus.in/bus-tickets/jodhpur-to-b...
8,Jaipur (Rajasthan) to Pilani,https://www.redbus.in/bus-tickets/jaipur-to-pi...
9,Kishangarh to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Rajasthan.csv"
df_RJ.to_csv(path,index=False)

In [ ]:
# Define lists to store the bus details

def  scroll():
    x=0
    l=[]
    while True:
        x=x+1
        driver_4.execute_script('scrollBy(0,100)')
        time.sleep(0.5)
        if(x>1000):
            break

wait = WebDriverWait(driver_4,20)

Rajasthan_bus_data=[]

def scrape_data(Rajasthan_link):

    # Open the route link
    driver_4.get(Rajasthan_link)
    time.sleep(3)
    Busroute=driver_4.find_element(By.XPATH,"//h1[@class='D136_h1']").text
    # Click to view bus details
    try:
        b=driver_4.find_elements(By.CSS_SELECTOR,"div[class='button']")
        for i in range(len(b)-1,-1,-1):
            b[i].click()

        # Scroll to the bottom of the page
        scroll()

        bus_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
        for bus in bus_items:
            try:
                busname = bus.find_element(By.CLASS_NAME, "travels").text
                bustype = bus.find_element(By.CLASS_NAME, "bus-type").text
                departing_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                duration = bus.find_element(By.CLASS_NAME, "dur").text
                reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text
                star_rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                price = bus.find_element(By.CSS_SELECTOR, ".seat-fare .fare span").text
                seats_available = bus.find_element(By.CLASS_NAME, "seat-left").text

                Rajasthan_bus_data.append({
                    "Busname": busname,
                    "Bustype": bustype,
                    "Busroutes": Busroute,
                    "Departing_time": departing_time,
                    "Duration": duration,
                    "Reaching_time": reaching_time,
                    "Star_rating": star_rating,
                    "Price": price,
                    "Seats_available": seats_available,
                    "buslink": Rajasthan_link
                })

            except Exception as e:
                print(f"An error occurred while scraping the data: {e}")
    except Exception as e:
        print(f"An error occurred while trying to click the view buttons: {e}")

for i in Rajasthan_link:
    scrape_data(i)
    time.sleep(3)

print("Data loaded successfully")

An error occurred while scraping the data: Message: no such element: Unable to locate element: {"method":"css selector","selector":".rating-sec .rating span"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF728B0B125+29573]
	(No symbol) [0x00007FF728A7FF50]
	(No symbol) [0x00007FF72893B6EA]
	(No symbol) [0x00007FF72898F815]
	(No symbol) [0x00007FF72898FA6C]
	(No symbol) [0x00007FF72898312C]
	(No symbol) [0x00007FF7289B733F]
	(No symbol) [0x00007FF728982FF6]
	(No symbol) [0x00007FF7289B7510]
	(No symbol) [0x00007FF7289D86BC]
	(No symbol) [0x00007FF7289B70A3]
	(No symbol) [0x00007FF7289812DF]
	(No symbol) [0x00007FF728982441]
	GetHandleVerifier [0x00007FF728E3C76D+3377613]
	GetHandleVerifier [0x00007FF728E87B67+3685831]
	GetHandleVerifier [0x00007FF728E7CF8B+3641835]
	GetHandleVerifier [0x00007FF728BCB2A6+816390]

In [ ]:
df_buses_Rajasthan = pd.DataFrame(Rajasthan_bus_data)
df_buses_Rajasthan

,Busname,Bustype,Busroutes,Departing_time,Duration,Reaching_time,Star_rating,Price,Seats_available,buslink
0,RSRTC - 193271,Express Non AC Seater 2+3,Udaipur to Jodhpur Bus,19:30,06h 30m,02:00,4.2,294,27 Seats available,https://www.redbus.in/bus-tickets/udaipur-to-j...
1,RSRTC - 189622,Express Non AC Seater 2+3,Udaipur to Jodhpur Bus,22:00,07h 00m,05:00,3.5,294,42 Seats available,https://www.redbus.in/bus-tickets/udaipur-to-j...
2,M R Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),Udaipur to Jodhpur Bus,17:50,04h 05m,21:55,4.5,600,38 Seats available,https://www.redbus.in/bus-tickets/udaipur-to-j...
3,Raj Travel,A/C Sleeper (2+1),Udaipur to Jodhpur Bus,22:30,07h 00m,05:30,4.6,560,13 Seats available,https://www.redbus.in/bus-tickets/udaipur-to-j...
4,Jain travels regd,A/C Seater / Sleeper (2+1),Udaipur to Jodhpur Bus,22:15,06h 15m,04:30,4.1,460,28 Seats available,https://www.redbus.in/bus-tickets/udaipur-to-j...
...,...,...,...,...,...,...,...,...,...,...
584,A One Travel Agency,A/C Sleeper (2+1),Jaipur to Kota(Rajasthan) Bus,21:15,04h 55m,02:10,1.6,899,34 Seats available,https://www.redbus.in/bus-tickets/jaipur-to-ko...
585,A One Travel Agency,Non A/C Seater / Sleeper (2+1),Jaipur to Kota(Rajasthan) Bus,21:00,05h 30m,02:30,1.6,350,39 Seats available,https://www.redbus.in/bus-tickets/jaipur-to-ko...
586,A One Travel Agency,Non A/C Seater / Sleeper (2+1),Jaipur to Kota(Rajasthan) Bus,21:00,05h 30m,02:30,1.6,350,39 Seats available,https://www.redbus.in/bus-tickets/jaipur-to-ko...
587,A One Travel Agency,NON A/C Seater/ Sleeper (2+1),Jaipur to Kota(Rajasthan) Bus,19:30,04h 44m,00:14,1.6,600,47 Seats available,https://www.redbus.in/bus-tickets/jaipur-to-ko...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Bus_Rajasthan.csv"
df_buses_Rajasthan.to_csv(path,index=False)

# 5.South Bengal

In [ ]:

# Open the browser
driver_5 = webdriver.Chrome()

# Load the webpage
driver_5.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile")

time.sleep(2)  # Wait for the page to load

driver_5.maximize_window()

# Retrieve bus links and routes
wait = WebDriverWait(driver_5, 10)

def South_Bengal_link_route(path):
    South_Bengal_link = []
    South_Bengal_route = []

    # Retrieve the route links
    for i in range(1, 6):
        paths = driver_5.find_elements(By.XPATH, path)

        for link in paths:
            d = link.get_attribute("href")
            South_Bengal_link.append(d)

        # Retrieve names of the routes
        for route in paths:
            South_Bengal_route.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_button = pagination.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()={i+1}]')
            time.sleep(2)

            # Retry clicking if intercepted
            retries = 6
            while retries > 0:
                try:
                    next_button.click()
                    break
                except ElementClickInterceptedException:
                    time.sleep(1)
                    retries -= 1
                    if retries == 0:
                        raise

        except NoSuchElementException:
            print(f"Total number of pages found {i}")
            break

    return South_Bengal_link, South_Bengal_route

South_Bengal_link, South_Bengal_route = South_Bengal_link_route("//a[@class='route']")

Total number of pages found 5


In [ ]:
df_SB=pd.DataFrame({"Route_name":South_Bengal_route,"Route_link":South_Bengal_link})
df_SB

,Route_name,Route_link
0,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...
1,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...
2,Haldia to Calcutta,https://www.redbus.in/bus-tickets/haldia-to-ko...
3,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
4,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...
5,Kolkata to Arambagh (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
6,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
7,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
8,Digha to Calcutta,https://www.redbus.in/bus-tickets/digha-to-kol...
9,Kolkata to Bankura,https://www.redbus.in/bus-tickets/kolkata-to-b...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/South Bengal.csv"
df_SB.to_csv(path,index=False)

In [ ]:
# Define lists to store the bus details

def  scroll():
    x=0
    l=[]
    while True:
        x=x+1
        driver_5.execute_script('scrollBy(0,100)')
        time.sleep(0.5)
        if(x>1000):
            break

wait = WebDriverWait(driver_5,20)

South_Bengal_bus_data=[]

def scrape_data(South_Bengal_link):

    # Open the route link
    driver_5.get(South_Bengal_link)
    time.sleep(3)
    Busroute=driver_5.find_element(By.XPATH,"//h1[@class='D136_h1']").text
    # Click to view bus details
    try:
        b=driver_5.find_elements(By.CSS_SELECTOR,"div[class='button']")
        for i in range(len(b)-1,-1,-1):
            b[i].click()

        # Scroll to the bottom of the page
        scroll()

        bus_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
        for bus in bus_items:
            try:
                busname = bus.find_element(By.CLASS_NAME, "travels").text
                bustype = bus.find_element(By.CLASS_NAME, "bus-type").text
                departing_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                duration = bus.find_element(By.CLASS_NAME, "dur").text
                reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text
                star_rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                price = bus.find_element(By.CSS_SELECTOR, ".seat-fare .fare span").text
                seats_available = bus.find_element(By.CLASS_NAME, "seat-left").text

                South_Bengal_bus_data.append({
                    "Busname": busname,
                    "Bustype": bustype,
                    "Busroutes": Busroute,
                    "Departing_time": departing_time,
                    "Duration": duration,
                    "Reaching_time": reaching_time,
                    "Star_rating": star_rating,
                    "Price": price,
                    "Seats_available": seats_available,
                    "buslink": South_Bengal_link
                })

            except Exception as e:
                print(f"An error occurred while scraping the data: {e}")
    except Exception as e:
        print(f"An error occurred while trying to click the view buttons: {e}")

for i in South_Bengal_link:
    scrape_data(i)
    time.sleep(3)

print("Data loaded successfully")

An error occurred while scraping the data: Message: no such element: Unable to locate element: {"method":"css selector","selector":".rating-sec .rating span"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF728B0B125+29573]
	(No symbol) [0x00007FF728A7FF50]
	(No symbol) [0x00007FF72893B6EA]
	(No symbol) [0x00007FF72898F815]
	(No symbol) [0x00007FF72898FA6C]
	(No symbol) [0x00007FF72898312C]
	(No symbol) [0x00007FF7289B733F]
	(No symbol) [0x00007FF728982FF6]
	(No symbol) [0x00007FF7289B7510]
	(No symbol) [0x00007FF7289D86BC]
	(No symbol) [0x00007FF7289B70A3]
	(No symbol) [0x00007FF7289812DF]
	(No symbol) [0x00007FF728982441]
	GetHandleVerifier [0x00007FF728E3C76D+3377613]
	GetHandleVerifier [0x00007FF728E87B67+3685831]
	GetHandleVerifier [0x00007FF728E7CF8B+3641835]
	GetHandleVerifier [0x00007FF728BCB2A6+816390]

In [ ]:
df_buses_South_Bengal = pd.DataFrame(South_Bengal_bus_data)
df_buses_South_Bengal

,Busname,Bustype,Busroutes,Departing_time,Duration,Reaching_time,Star_rating,Price,Seats_available,buslink
0,SBSTC-DURGAPUR - KOLKATA - 05:00 (DGP DEPOT) -...,Non AC Seater (2+3),Durgapur (West Bengal) to Kolkata Bus,05:00,03h 30m,08:30,4.1,150,34 Seats available,https://www.redbus.in/bus-tickets/durgapur-to-...
1,SBSTC-DURGAPUR - KARUNAMOYEE - 05:15 (DGP DEPO...,Non AC Seater (2+3),Durgapur (West Bengal) to Kolkata Bus,05:15,04h 00m,09:15,4.1,155,13 Seats available,https://www.redbus.in/bus-tickets/durgapur-to-...
2,SBSTC-DURGAPUR - KOLKATA - 05:30 (DGP DEPOT) -...,Non AC Seater (2+3),Durgapur (West Bengal) to Kolkata Bus,05:30,18h 30m,00:00,3.4,150,50 Seats available,https://www.redbus.in/bus-tickets/durgapur-to-...
3,SBSTC-DURGAPUR - KOLKATA - L/S - 06:00 (DGP DE...,Non AC Seater (2+3),Durgapur (West Bengal) to Kolkata Bus,06:00,03h 30m,09:30,3.2,150,48 Seats available,https://www.redbus.in/bus-tickets/durgapur-to-...
4,SBSTC-DURGAPUR - KARUNAMOYEE - 06:15 (DGP DEPO...,Non AC Seater (2+3),Durgapur (West Bengal) to Kolkata Bus,06:15,04h 00m,10:15,3.6,155,37 Seats available,https://www.redbus.in/bus-tickets/durgapur-to-...
...,...,...,...,...,...,...,...,...,...,...
1560,Aradhana Bus Service,A/C Seater / Sleeper (2+2),Barasat (West Bengal) to Digha Bus,02:30,05h 10m,07:40,2.4,500,43 Seats available,https://www.redbus.in/bus-tickets/barasat-west...
1561,Ankita Paribahan (Satya),A/C Seater Push Back (2+3),Barasat (West Bengal) to Digha Bus,05:40,05h 55m,11:35,2.7,420,21 Seats available,https://www.redbus.in/bus-tickets/barasat-west...
1562,Snemita Parisheba,AC Seater (2+3),Barasat (West Bengal) to Digha Bus,05:50,04h 30m,10:20,1.0,420,21 Seats available,https://www.redbus.in/bus-tickets/barasat-west...
1563,Rudraksha City Service,A/C Seater (2+3),Barasat (West Bengal) to Digha Bus,07:00,05h 20m,12:20,2.7,350,36 Seats available,https://www.redbus.in/bus-tickets/barasat-west...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Bus_South Bengal.csv"
df_buses_South_Bengal.to_csv(path,index=False)

In [ ]:
SouthBengal = pd.read_csv("Bus_South Bengal.csv")
bus_SouthBengal = SouthBengal.dropna()
bus_SouthBengal

,Busname,Bustype,Busroutes,Departing_time,Duration,Reaching_time,Star_rating,Price,Seats_available,buslink
0,SBSTC-DURGAPUR - KOLKATA - 05:00 (DGP DEPOT) -...,Non AC Seater (2+3),Durgapur (West Bengal) to Kolkata Bus,05:00,03h 30m,08:30,4.1,150.0,34 Seats available,https://www.redbus.in/bus-tickets/durgapur-to-...
1,SBSTC-DURGAPUR - KARUNAMOYEE - 05:15 (DGP DEPO...,Non AC Seater (2+3),Durgapur (West Bengal) to Kolkata Bus,05:15,04h 00m,09:15,4.1,155.0,13 Seats available,https://www.redbus.in/bus-tickets/durgapur-to-...
2,SBSTC-DURGAPUR - KOLKATA - 05:30 (DGP DEPOT) -...,Non AC Seater (2+3),Durgapur (West Bengal) to Kolkata Bus,05:30,18h 30m,00:00,3.4,150.0,50 Seats available,https://www.redbus.in/bus-tickets/durgapur-to-...
3,SBSTC-DURGAPUR - KOLKATA - L/S - 06:00 (DGP DE...,Non AC Seater (2+3),Durgapur (West Bengal) to Kolkata Bus,06:00,03h 30m,09:30,3.2,150.0,48 Seats available,https://www.redbus.in/bus-tickets/durgapur-to-...
4,SBSTC-DURGAPUR - KARUNAMOYEE - 06:15 (DGP DEPO...,Non AC Seater (2+3),Durgapur (West Bengal) to Kolkata Bus,06:15,04h 00m,10:15,3.6,155.0,37 Seats available,https://www.redbus.in/bus-tickets/durgapur-to-...
...,...,...,...,...,...,...,...,...,...,...
1560,Aradhana Bus Service,A/C Seater / Sleeper (2+2),Barasat (West Bengal) to Digha Bus,02:30,05h 10m,07:40,2.4,500.0,43 Seats available,https://www.redbus.in/bus-tickets/barasat-west...
1561,Ankita Paribahan (Satya),A/C Seater Push Back (2+3),Barasat (West Bengal) to Digha Bus,05:40,05h 55m,11:35,2.7,420.0,21 Seats available,https://www.redbus.in/bus-tickets/barasat-west...
1562,Snemita Parisheba,AC Seater (2+3),Barasat (West Bengal) to Digha Bus,05:50,04h 30m,10:20,1.0,420.0,21 Seats available,https://www.redbus.in/bus-tickets/barasat-west...
1563,Rudraksha City Service,A/C Seater (2+3),Barasat (West Bengal) to Digha Bus,07:00,05h 20m,12:20,2.7,350.0,36 Seats available,https://www.redbus.in/bus-tickets/barasat-west...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Bus_South Bengal.csv"
bus_SouthBengal.to_csv(path,index=False)

# 6.Himachal Pradesh

In [ ]:

# Open the browser
driver_6 = webdriver.Chrome()

# Load the webpage
driver_6.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")

time.sleep(2)  # Wait for the page to load

driver_6.maximize_window()

# Retrieve bus links and routes
wait = WebDriverWait(driver_6, 10)

def Himachal_link_route(path):
    Himachal_link = []
    Himachal_route = []

    # Retrieve the route links
    for i in range(1, 5):
        paths = driver_6.find_elements(By.XPATH, path)

        for link in paths:
            d = link.get_attribute("href")
            Himachal_link.append(d)

        # Retrieve names of the routes
        for route in paths:
            Himachal_route.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_button = pagination.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()={i+1}]')
            time.sleep(2)

            # Retry clicking if intercepted
            retries = 5
            while retries > 0:
                try:
                    next_button.click()
                    break
                except ElementClickInterceptedException:
                    time.sleep(1)
                    retries -= 1
                    if retries == 0:
                        raise

        except NoSuchElementException:
            print(f"Total number of pages found {i}")
            break

    return Himachal_link, Himachal_route

Himachal_link, Himachal_route = Himachal_link_route("//a[@class='route']")

Total number of pages found 4


In [ ]:
df_HP=pd.DataFrame({"Route_name":Himachal_route,"Route_link":Himachal_link})
df_HP

,Route_name,Route_link
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
2,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
3,Shimla to Delhi,https://www.redbus.in/bus-tickets/shimla-to-delhi
4,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
5,Hamirpur (Himachal Pradesh) to Delhi,https://www.redbus.in/bus-tickets/hamirpur-him...
6,Chamba (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/chamba-himac...
7,Delhi to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-ham...
8,Chandigarh to Dharamshala (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
9,Delhi to Chamba (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-cha...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Himachal.csv"
df_HP.to_csv(path,index=False)

In [ ]:
# Define lists to store the bus details

def  scroll():
    x=0
    l=[]
    while True:
        x=x+1
        driver_6.execute_script('scrollBy(0,100)')
        time.sleep(0.5)
        if(x>1000):
            break

wait = WebDriverWait(driver_6,20)

Himachal_bus_data=[]

def scrape_data(Himachal_link):

    # Open the route link
    driver_6.get(Himachal_link)
    time.sleep(3)
    Busroute=driver_6.find_element(By.XPATH,"//h1[@class='D136_h1']").text
    # Click to view bus details
    try:
        b=driver_6.find_elements(By.CSS_SELECTOR,"div[class='button']")
        for i in range(len(b)-1,-1,-1):
            b[i].click()

        # Scroll to the bottom of the page
        scroll()

        bus_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
        for bus in bus_items:
            try:
                busname = bus.find_element(By.CLASS_NAME, "travels").text
                bustype = bus.find_element(By.CLASS_NAME, "bus-type").text
                departing_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                duration = bus.find_element(By.CLASS_NAME, "dur").text
                reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text
                star_rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                price = bus.find_element(By.CSS_SELECTOR, ".seat-fare .fare span").text
                seats_available = bus.find_element(By.CLASS_NAME, "seat-left").text

                Himachal_bus_data.append({
                    "Busname": busname,
                    "Bustype": bustype,
                    "Busroutes": Busroute,
                    "Departing_time": departing_time,
                    "Duration": duration,
                    "Reaching_time": reaching_time,
                    "Star_rating": star_rating,
                    "Price": price,
                    "Seats_available": seats_available,
                    "buslink": Himachal_link
                })

            except Exception as e:
                print(f"An error occurred while scraping the data: {e}")
    except Exception as e:
        print(f"An error occurred while trying to click the view buttons: {e}")

for i in Himachal_link:
    scrape_data(i)
    time.sleep(3)

print("Data loaded successfully")

An error occurred while scraping the data: Message: no such element: Unable to locate element: {"method":"css selector","selector":".rating-sec .rating span"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7138DB125+29573]
	(No symbol) [0x00007FF71384FF50]
	(No symbol) [0x00007FF71370B6EA]
	(No symbol) [0x00007FF71375F815]
	(No symbol) [0x00007FF71375FA6C]
	(No symbol) [0x00007FF71375312C]
	(No symbol) [0x00007FF71378733F]
	(No symbol) [0x00007FF713752FF6]
	(No symbol) [0x00007FF713787510]
	(No symbol) [0x00007FF7137A86BC]
	(No symbol) [0x00007FF7137870A3]
	(No symbol) [0x00007FF7137512DF]
	(No symbol) [0x00007FF713752441]
	GetHandleVerifier [0x00007FF713C0C76D+3377613]
	GetHandleVerifier [0x00007FF713C57B67+3685831]
	GetHandleVerifier [0x00007FF713C4CF8B+3641835]
	GetHandleVerifier [0x00007FF71399B2A6+816390]

In [ ]:
df_buses_Himachal = pd.DataFrame(Himachal_bus_data)
df_buses_Himachal

,Busname,Bustype,Busroutes,Departing_time,Duration,Reaching_time,Star_rating,Price,Seats_available,buslink
0,HRTC - 6,Himsuta AC Seater Volvo/Scania 2+2,Delhi to Shimla Bus,06:45,09h 25m,16:10,4.5,912,34 Seats available,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,HRTC - 592,A/C Executive (2+3),Delhi to Shimla Bus,08:05,10h 05m,18:10,2.7,632,40 Seats available,https://www.redbus.in/bus-tickets/delhi-to-shimla
2,HRTC - 129,Ordinary 3+2 Non AC Seater,Delhi to Shimla Bus,08:50,09h 50m,18:40,2.6,512,37 Seats available,https://www.redbus.in/bus-tickets/delhi-to-shimla
3,HRTC - 7,Himsuta AC Seater Volvo/Scania 2+2,Delhi to Shimla Bus,09:25,09h 05m,18:30,4.6,912,21 Seats available,https://www.redbus.in/bus-tickets/delhi-to-shimla
4,HRTC - 657,Ordinary 3+2 Non AC Seater,Delhi to Shimla Bus,11:00,11h 00m,22:00,2.7,512,33 Seats available,https://www.redbus.in/bus-tickets/delhi-to-shimla
...,...,...,...,...,...,...,...,...,...,...
1091,SIDDHI VINAYAK TRAVELS,Volvo A/C Semi Sleeper (2+2),Manali to Delhi Bus,18:00,13h 00m,07:00,1.0,1139,5 Seats available,https://www.redbus.in/bus-tickets/manali-to-delhi
1092,Heera Himachal Holidays,Volvo A/C Semi Sleeper (2+2),Manali to Delhi Bus,19:30,12h 30m,08:00,2.9,999,31 Seats available,https://www.redbus.in/bus-tickets/manali-to-delhi
1093,Yatrabybus.com,Bharat Benz A/C Semi Sleeper (2+2),Manali to Delhi Bus,18:00,10h 00m,04:00,2.5,1199,49 Seats available,https://www.redbus.in/bus-tickets/manali-to-delhi
1094,Royal Travels Hills,Volvo A/C Semi Sleeper (2+2),Manali to Delhi Bus,18:00,12h 00m,06:00,4.5,1099,29 Seats available,https://www.redbus.in/bus-tickets/manali-to-delhi


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Bus_Himachal.csv"
df_buses_Himachal.to_csv(path,index=False)

# 7.Assam

In [ ]:

# Open the browser
driver_7 = webdriver.Chrome()

# Load the webpage
driver_7.get("https://www.redbus.in/online-booking/astc/?utm_source=rtchometile")

time.sleep(3)  # Wait for the page to load

driver_7.maximize_window()

# Retrieve bus links and routes
wait = WebDriverWait(driver_7, 20)

def Assam_link_route(path):
    Assam_link = []
    Assam_route = []

    # Retrieve the route links
    for i in range(1, 6):
        paths = driver_7.find_elements(By.XPATH, path)

        for link in paths:
            d = link.get_attribute("href")
            Assam_link.append(d)

        # Retrieve names of the routes
        for route in paths:
            Assam_route.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_button = pagination.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()={i+1}]')
            time.sleep(3)

            # Retry clicking if intercepted
            retries = 6
            while retries > 0:
                try:
                    next_button.click()
                    break
                except ElementClickInterceptedException:
                    time.sleep(1)
                    retries -= 1
                    if retries == 0:
                        raise

        except NoSuchElementException:
            print(f"Total number of pages found {i}")
            break

    return Assam_link, Assam_route

Assam_link, Assam_route = Assam_link_route("//a[@class='route']")

Total number of pages found 5


In [ ]:
df_AS=pd.DataFrame({"Route_name":Assam_route,"Route_link":Assam_link})
df_AS

,Route_name,Route_link
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
2,Guwahati to Nagaon (Assam),https://www.redbus.in/bus-tickets/guwahati-to-...
3,Nagaon (Assam) to Guwahati,https://www.redbus.in/bus-tickets/nagaon-to-gu...
4,Goalpara to Guwahati,https://www.redbus.in/bus-tickets/goalpara-to-...
5,Jorhat to North Lakhimpur,https://www.redbus.in/bus-tickets/jorhat-to-no...
6,Dhubri to Guwahati,https://www.redbus.in/bus-tickets/dhubri-to-gu...
7,Jorhat to Dibrugarh,https://www.redbus.in/bus-tickets/jorhat-to-di...
8,North Lakhimpur to Jorhat,https://www.redbus.in/bus-tickets/north-lakhim...
9,North Lakhimpur to Sibsagar,https://www.redbus.in/bus-tickets/north-lakhim...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Assam.csv"
df_AS.to_csv(path,index=False)

In [ ]:
# Define lists to store the bus details

def  scroll():
    x=0
    l=[]
    while True:
        x=x+1
        driver_7.execute_script('scrollBy(0,100)')
        time.sleep(0.5)
        if(x>1000):
            break

wait = WebDriverWait(driver_7,20)

Assam_bus_data=[]

def scrape_data(Assam_link):

    # Open the route link
    driver_7.get(Assam_link)
    time.sleep(3)
    Busroute=driver_7.find_element(By.XPATH,"//h1[@class='D136_h1']").text
    # Click to view bus details
    try:
        b=driver_7.find_elements(By.CSS_SELECTOR,"div[class='button']")
        for i in range(len(b)-1,-1,-1):
            b[i].click()

        # Scroll to the bottom of the page
        scroll()

        bus_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
        for bus in bus_items:
            try:
                busname = bus.find_element(By.CLASS_NAME, "travels").text
                bustype = bus.find_element(By.CLASS_NAME, "bus-type").text
                departing_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                duration = bus.find_element(By.CLASS_NAME, "dur").text
                reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text
                star_rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                price = bus.find_element(By.CSS_SELECTOR, ".seat-fare .fare span").text
                seats_available = bus.find_element(By.CLASS_NAME, "seat-left").text

                Assam_bus_data.append({
                    "Busname": busname,
                    "Bustype": bustype,
                    "Busroutes": Busroute,
                    "Departing_time": departing_time,
                    "Duration": duration,
                    "Reaching_time": reaching_time,
                    "Star_rating": star_rating,
                    "Price": price,
                    "Seats_available": seats_available,
                    "buslink": Assam_link
                })

            except Exception as e:
                print(f"An error occurred while scraping the data: {e}")
    except Exception as e:
        print(f"An error occurred while trying to click the view buttons: {e}")

for i in Assam_link:
    scrape_data(i)
    time.sleep(3)

print("Data loaded successfully")

An error occurred while trying to click the view buttons: Message: 

An error occurred while scraping the data: Message: no such element: Unable to locate element: {"method":"css selector","selector":".rating-sec .rating span"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF79D1CB645+29573]
	(No symbol) [0x00007FF79D140470]
	(No symbol) [0x00007FF79CFFB6EA]
	(No symbol) [0x00007FF79D04F815]
	(No symbol) [0x00007FF79D04FA6C]
	(No symbol) [0x00007FF79D04312C]
	(No symbol) [0x00007FF79D07733F]
	(No symbol) [0x00007FF79D042FF6]
	(No symbol) [0x00007FF79D077510]
	(No symbol) [0x00007FF79D0986BC]
	(No symbol) [0x00007FF79D0770A3]
	(No symbol) [0x00007FF79D0412DF]
	(No symbol) [0x00007FF79D042441]
	GetHandleVerifier [0x00007FF79D4FC58D+3375821]
	GetHandleVerifier [0x00007FF79D547987+3684039]
	GetHandleVerifier [0x000

In [ ]:
Assam = pd.DataFrame(Assam_bus_data)
Assam

,Busname,Bustype,Busroutes,Departing_time,Duration,Reaching_time,Star_rating,Price,Seats_available,buslink
0,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),Tezpur to Guwahati Bus,21:30,06h 00m,03:30,3.0,400,20 Seats available,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),Tezpur to Guwahati Bus,22:15,05h 30m,03:45,3.5,450,19 Seats available,https://www.redbus.in/bus-tickets/tezpur-to-gu...
2,Himalayan Travels,NON AC Seater / Sleeper 2+1,Tezpur to Guwahati Bus,23:40,04h 20m,04:00,2.2,500,9 Seats available,https://www.redbus.in/bus-tickets/tezpur-to-gu...
3,Yatra Travels,NON AC Seater/ Sleeper (2+1),Guwahati to Tezpur Bus,20:00,04h 00m,00:00,3.5,350,12 Seats available,https://www.redbus.in/bus-tickets/guwahati-to-...
4,Yatra Travels,NON AC Seater/ Sleeper (2+1),Guwahati to Tezpur Bus,20:30,04h 30m,01:00,4.2,350,16 Seats available,https://www.redbus.in/bus-tickets/guwahati-to-...
...,...,...,...,...,...,...,...,...,...,...
323,ZAMZAM TRAVELS,A/C Seater (2+1),Guwahati to Golaghat Bus,13:05,07h 25m,20:30,3.9,550,36 Seats available,https://www.redbus.in/bus-tickets/guwahati-to-...
324,Assam State Transport Corporation (ASTC) - 142178,Bharat Benz A/C Seater (2+2),Jorhat to Gogamukh Bus,06:45,05h 15m,12:00,3.8,339,15 Seats available,https://www.redbus.in/bus-tickets/jorhat-to-go...
325,Assam State Transport Corporation (ASTC) - 174758,NON A/C Seater Push Back (2+1),Dibrugarh to Biswanath Charali Bus,08:45,06h 15m,15:00,2.3,390,18 Seats available,https://www.redbus.in/bus-tickets/dibrugarh-to...
326,Air Bus Travels,NON AC Seater / Sleeper 2+1,Dibrugarh to Biswanath Charali Bus,19:40,09h 20m,05:00,3.1,600,46 Seats available,https://www.redbus.in/bus-tickets/dibrugarh-to...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Bus_Assam_d.csv"
Assam.to_csv(path,index=False)

In [ ]:
bus_assam = pd.read_csv("Bus_Assam_d.csv")

In [ ]:
df_buses_Assam = bus_assam.dropna()
df_buses_Assam

,Busname,Bustype,Busroutes,Departing_time,Duration,Reaching_time,Star_rating,Price,Seats_available,buslink
0,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),Tezpur to Guwahati Bus,21:30,06h 00m,03:30,3.0,400.0,20 Seats available,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),Tezpur to Guwahati Bus,22:15,05h 30m,03:45,3.5,450.0,19 Seats available,https://www.redbus.in/bus-tickets/tezpur-to-gu...
2,Himalayan Travels,NON AC Seater / Sleeper 2+1,Tezpur to Guwahati Bus,23:40,04h 20m,04:00,2.2,500.0,9 Seats available,https://www.redbus.in/bus-tickets/tezpur-to-gu...
3,Yatra Travels,NON AC Seater/ Sleeper (2+1),Guwahati to Tezpur Bus,20:00,04h 00m,00:00,3.5,350.0,12 Seats available,https://www.redbus.in/bus-tickets/guwahati-to-...
4,Yatra Travels,NON AC Seater/ Sleeper (2+1),Guwahati to Tezpur Bus,20:30,04h 30m,01:00,4.2,350.0,16 Seats available,https://www.redbus.in/bus-tickets/guwahati-to-...
...,...,...,...,...,...,...,...,...,...,...
323,ZAMZAM TRAVELS,A/C Seater (2+1),Guwahati to Golaghat Bus,13:05,07h 25m,20:30,3.9,550.0,36 Seats available,https://www.redbus.in/bus-tickets/guwahati-to-...
324,Assam State Transport Corporation (ASTC) - 142178,Bharat Benz A/C Seater (2+2),Jorhat to Gogamukh Bus,06:45,05h 15m,12:00,3.8,339.0,15 Seats available,https://www.redbus.in/bus-tickets/jorhat-to-go...
325,Assam State Transport Corporation (ASTC) - 174758,NON A/C Seater Push Back (2+1),Dibrugarh to Biswanath Charali Bus,08:45,06h 15m,15:00,2.3,390.0,18 Seats available,https://www.redbus.in/bus-tickets/dibrugarh-to...
326,Air Bus Travels,NON AC Seater / Sleeper 2+1,Dibrugarh to Biswanath Charali Bus,19:40,09h 20m,05:00,3.1,600.0,46 Seats available,https://www.redbus.in/bus-tickets/dibrugarh-to...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Bus_Assam.csv"
df_buses_Assam.to_csv(path,index=False)

# 8.West Bengal

In [ ]:

# Open the browser
driver_8 = webdriver.Chrome()

# Load the webpage
driver_8.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile")

time.sleep(3)  # Wait for the page to load

driver_8.maximize_window()

# Retrieve bus links and routes
wait = WebDriverWait(driver_8, 20)

def West_Bengal_link_route(path):
    West_Bengal_link = []
    West_Bengal_route = []

    # Retrieve the route links
    for i in range(1, 5):
        paths = driver_8.find_elements(By.XPATH, path)

        for link in paths:
            d = link.get_attribute("href")
            West_Bengal_link.append(d)

        # Retrieve names of the routes
        for route in paths:
            West_Bengal_route.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_button = pagination.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()={i+1}]')
            time.sleep(3)

            # Retry clicking if intercepted
            retries = 5
            while retries > 0:
                try:
                    next_button.click()
                    break
                except ElementClickInterceptedException:
                    time.sleep(1)
                    retries -= 1
                    if retries == 0:
                        raise

        except NoSuchElementException:
            print(f"Total number of pages found {i}")
            break

    return West_Bengal_link, West_Bengal_route

West_Bengal_link, West_Bengal_route = West_Bengal_link_route("//a[@class='route']")

Total number of pages found 4


In [ ]:
df_WB=pd.DataFrame({"Route_name":West_Bengal_route,"Route_link":West_Bengal_link})
df_WB

,Route_name,Route_link
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
1,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...
2,Digha to Calcutta,https://www.redbus.in/bus-tickets/digha-to-kol...
3,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
4,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...
5,Kolkata to Suri,https://www.redbus.in/bus-tickets/kolkata-to-suri
6,Barasat (West Bengal) to Midnapore,https://www.redbus.in/bus-tickets/barasat-west...
7,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
8,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...
9,Barasat (West Bengal) to Contai (Kanthi),https://www.redbus.in/bus-tickets/barasat-west...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/West Bengal.csv"
df_WB.to_csv(path,index=False)

In [ ]:
# Define lists to store the bus details

def  scroll():
    x=0
    l=[]
    while True:
        x=x+1
        driver_8.execute_script('scrollBy(0,100)')
        time.sleep(0.5)
        if(x>1000):
            break

wait = WebDriverWait(driver_8,20)

West_Bengal_bus_data=[]

def scrape_data(West_Bengal_link):

    # Open the route link
    driver_8.get(West_Bengal_link)
    time.sleep(3)
    Busroute=driver_8.find_element(By.XPATH,"//h1[@class='D136_h1']").text
    # Click to view bus details
    try:
        b=driver_8.find_elements(By.CSS_SELECTOR,"div[class='button']")
        for i in range(len(b)-1,-1,-1):
            b[i].click()

        # Scroll to the bottom of the page
        scroll()

        bus_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
        for bus in bus_items:
            try:
                busname = bus.find_element(By.CLASS_NAME, "travels").text
                bustype = bus.find_element(By.CLASS_NAME, "bus-type").text
                departing_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                duration = bus.find_element(By.CLASS_NAME, "dur").text
                reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text
                star_rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                price = bus.find_element(By.CSS_SELECTOR, ".seat-fare .fare span").text
                seats_available = bus.find_element(By.CLASS_NAME, "seat-left").text

                West_Bengal_bus_data.append({
                    "Busname": busname,
                    "Bustype": bustype,
                    "Busroutes": Busroute,
                    "Departing_time": departing_time,
                    "Duration": duration,
                    "Reaching_time": reaching_time,
                    "Star_rating": star_rating,
                    "Price": price,
                    "Seats_available": seats_available,
                    "buslink": West_Bengal_link
                })

            except Exception as e:
                print(f"An error occurred while scraping the data: {e}")
    except Exception as e:
        print(f"An error occurred while trying to click the view buttons: {e}")

for i in West_Bengal_link:
    scrape_data(i)
    time.sleep(3)

print("Data loaded successfully")


An error occurred while trying to click the view buttons: Message: 

An error occurred while scraping the data: Message: no such element: Unable to locate element: {"method":"css selector","selector":".rating-sec .rating span"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7138DB125+29573]
	(No symbol) [0x00007FF71384FF50]
	(No symbol) [0x00007FF71370B6EA]
	(No symbol) [0x00007FF71375F815]
	(No symbol) [0x00007FF71375FA6C]
	(No symbol) [0x00007FF71375312C]
	(No symbol) [0x00007FF71378733F]
	(No symbol) [0x00007FF713752FF6]
	(No symbol) [0x00007FF713787510]
	(No symbol) [0x00007FF7137A86BC]
	(No symbol) [0x00007FF7137870A3]
	(No symbol) [0x00007FF7137512DF]
	(No symbol) [0x00007FF713752441]
	GetHandleVerifier [0x00007FF713C0C76D+3377613]
	GetHandleVerifier [0x00007FF713C57B67+3685831]
	GetHandleVerifier [0x000

In [ ]:
df_buses_West_Bengal = pd.DataFrame(West_Bengal_bus_data)
df_buses_West_Bengal

,Busname,Bustype,Busroutes,Departing_time,Duration,Reaching_time,Star_rating,Price,Seats_available,buslink
0,The Joy Ride,A/C Seater / Sleeper (2+2),Digha to Barasat (West Bengal) Bus,21:15,06h 20m,03:35,5.0,488,35 Seats available,https://www.redbus.in/bus-tickets/digha-to-bar...
1,Arunita Paribahan,A/C Seater (2+3),Digha to Barasat (West Bengal) Bus,23:35,05h 25m,05:00,5.0,370,21 Seats available,https://www.redbus.in/bus-tickets/digha-to-bar...
2,Satya Paribahan,A/C Seater (2+3),Digha to Barasat (West Bengal) Bus,23:00,06h 10m,05:10,2.8,280,31 Seats available,https://www.redbus.in/bus-tickets/digha-to-bar...
3,Aradhana Bus Service,A/C Seater / Sleeper (2+2),Digha to Barasat (West Bengal) Bus,23:00,05h 30m,04:30,2.1,550,71 Seats available,https://www.redbus.in/bus-tickets/digha-to-bar...
4,Snemita Paribahan (Shinjini),AC Seater (2+3),Digha to Barasat (West Bengal) Bus,23:35,05h 25m,05:00,2.4,420,27 Seats available,https://www.redbus.in/bus-tickets/digha-to-bar...
...,...,...,...,...,...,...,...,...,...,...
469,SBSTC-HABRA - DIGHA - VIA - NIBEDITA SETU - 04...,Non AC Seater (2+3),Habra to Nandakumar (west bengal) Bus,04:00,20h 00m,00:00,2.2,118,49 Seats available,https://www.redbus.in/bus-tickets/habra-to-nan...
470,SBSTC-HABRA - DIGHA - VIA - NIBEDITA SETU - 05...,Non AC Seater (2+3),Habra to Nandakumar (west bengal) Bus,05:40,18h 20m,00:00,2.6,118,50 Seats available,https://www.redbus.in/bus-tickets/habra-to-nan...
471,SBSTC-HABRA - HALDIA - 06:50 (HALDIA) - 1509,Non AC Seater (2+3),Habra to Nandakumar (west bengal) Bus,06:50,17h 10m,00:00,1.3,118,52 Seats available,https://www.redbus.in/bus-tickets/habra-to-nan...
472,WBTC (CTC) Habra-Midnapur - 194|14:45,Non AC Seater (2+3),Midnapore to Kolkata Airport Bus,14:45,02h 55m,17:40,3.9,111,45 Seats available,https://www.redbus.in/bus-tickets/midnapore-to...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Bus_West Bengal.csv"
df_buses_West_Bengal.to_csv(path,index=False)

# 9.Punjab

In [ ]:

# Open the browser
driver_9 = webdriver.Chrome()

# Load the webpage
driver_9.get("https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile")

time.sleep(3)  # Wait for the page to load

driver_9.maximize_window()

# Retrieve bus links and routes
wait = WebDriverWait(driver_9, 20)

def Punjab_link_route(path):
    Punjab_link = []
    Punjab_route = []

    # Retrieve the route links
    for i in range(1, 3):
        paths = driver_9.find_elements(By.XPATH, path)

        for link in paths:
            d = link.get_attribute("href")
            Punjab_link.append(d)

        # Retrieve names of the routes
        for route in paths:
            Punjab_route.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_button = pagination.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()={i+1}]')
            time.sleep(3)

            # Retry clicking if intercepted
            retries = 3
            while retries > 0:
                try:
                    next_button.click()
                    break
                except ElementClickInterceptedException:
                    time.sleep(1)
                    retries -= 1
                    if retries == 0:
                        raise

        except NoSuchElementException:
            print(f"Total number of pages found {i}")
            break

    return Punjab_link, Punjab_route

Punjab_link, Punjab_route = Punjab_link_route("//a[@class='route']")

Total number of pages found 2


In [ ]:
df_PJ=pd.DataFrame({"Route_name":Punjab_route,"Route_link":Punjab_link})
df_PJ

,Route_name,Route_link
0,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
1,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...
2,Delhi to Ludhiana,https://www.redbus.in/bus-tickets/delhi-to-lud...
3,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...
4,Jalandhar to Delhi,https://www.redbus.in/bus-tickets/jalandhar-to...
5,Delhi to Jalandhar,https://www.redbus.in/bus-tickets/delhi-to-jal...
6,Delhi Airport to Patiala,https://www.redbus.in/bus-tickets/delhi-airpor...
7,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...
8,Delhi Airport to Ludhiana,https://www.redbus.in/bus-tickets/delhi-airpor...
9,Jalandhar to Delhi Airport,https://www.redbus.in/bus-tickets/jalandhar-to...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Punjab.csv"
df_PJ.to_csv(path,index=False)

In [ ]:
# Define lists to store the bus details

def  scroll():
    x=0
    l=[]
    while True:
        x=x+1
        driver_9.execute_script('scrollBy(0,100)')
        time.sleep(0.5)
        if(x>1000):
            break

wait = WebDriverWait(driver_9,20)

Punjab_bus_data=[]

def scrape_data(Punjab_link):

    # Open the route link
    driver_9.get(Punjab_link)
    time.sleep(3)
    Busroute=driver_9.find_element(By.XPATH,"//h1[@class='D136_h1']").text
    # Click to view bus details
    try:
        b=driver_9.find_elements(By.CSS_SELECTOR,"div[class='button']")
        for i in range(len(b)-1,-1,-1):
            b[i].click()

        # Scroll to the bottom of the page
        scroll()

        bus_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
        for bus in bus_items:
            try:
                busname = bus.find_element(By.CLASS_NAME, "travels").text
                bustype = bus.find_element(By.CLASS_NAME, "bus-type").text
                departing_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                duration = bus.find_element(By.CLASS_NAME, "dur").text
                reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text
                star_rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                price = bus.find_element(By.CSS_SELECTOR, ".seat-fare .fare span").text
                seats_available = bus.find_element(By.CLASS_NAME, "seat-left").text

                Punjab_bus_data.append({
                    "Busname": busname,
                    "Bustype": bustype,
                    "Busroutes": Busroute,
                    "Departing_time": departing_time,
                    "Duration": duration,
                    "Reaching_time": reaching_time,
                    "Star_rating": star_rating,
                    "Price": price,
                    "Seats_available": seats_available,
                    "buslink": Punjab_link
                })

            except Exception as e:
                print(f"An error occurred while scraping the data: {e}")
    except Exception as e:
        print(f"An error occurred while trying to click the view buttons: {e}")

for i in Punjab_link:
    scrape_data(i)
    time.sleep(3)

print("Data loaded successfully")

An error occurred while scraping the data: Message: no such element: Unable to locate element: {"method":"css selector","selector":".rating-sec .rating span"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF75796B125+29573]
	(No symbol) [0x00007FF7578DFF50]
	(No symbol) [0x00007FF75779B6EA]
	(No symbol) [0x00007FF7577EF815]
	(No symbol) [0x00007FF7577EFA6C]
	(No symbol) [0x00007FF7577E312C]
	(No symbol) [0x00007FF75781733F]
	(No symbol) [0x00007FF7577E2FF6]
	(No symbol) [0x00007FF757817510]
	(No symbol) [0x00007FF7578386BC]
	(No symbol) [0x00007FF7578170A3]
	(No symbol) [0x00007FF7577E12DF]
	(No symbol) [0x00007FF7577E2441]
	GetHandleVerifier [0x00007FF757C9C74D+3377581]
	GetHandleVerifier [0x00007FF757CE7B47+3685799]
	GetHandleVerifier [0x00007FF757CDCF6B+3641803]
	GetHandleVerifier [0x00007FF757A2B2A6+816390]

In [ ]:
df_buses_Punjab = pd.DataFrame(Punjab_bus_data)
df_buses_Punjab

,Busname,Bustype,Busroutes,Departing_time,Duration,Reaching_time,Star_rating,Price,Seats_available,buslink
0,PEPSU (Punjab) - 67085,Volvo AC Seater (2+2),Delhi to Patiala Bus,03:00,05h 00m,08:00,4.5,711,38 Seats available,https://www.redbus.in/bus-tickets/delhi-to-pat...
1,PEPSU (Punjab) - 67070,Volvo AC Seater (2+2),Delhi to Patiala Bus,06:15,05h 30m,11:45,3.6,711,37 Seats available,https://www.redbus.in/bus-tickets/delhi-to-pat...
2,PEPSU (Punjab) - 67069,Volvo AC Seater (2+2),Delhi to Patiala Bus,08:00,05h 30m,13:30,4.3,711,36 Seats available,https://www.redbus.in/bus-tickets/delhi-to-pat...
3,PEPSU (Punjab) - 67105,AC Seater Hvac 2+2,Delhi to Patiala Bus,09:50,05h 30m,15:20,2.7,454,36 Seats available,https://www.redbus.in/bus-tickets/delhi-to-pat...
4,PEPSU (Punjab) - 67160,Volvo AC Seater (2+2),Delhi to Patiala Bus,11:46,05h 29m,17:15,4.2,711,39 Seats available,https://www.redbus.in/bus-tickets/delhi-to-pat...
...,...,...,...,...,...,...,...,...,...,...
583,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),Delhi Airport to Phagwara Bus,23:30,11h 15m,10:45,2.9,2500,44 Seats available,https://www.redbus.in/bus-tickets/delhi-airpor...
584,Indo Canadian Transport Pvt. Ltd.,A/C Sleeper (2+1),Delhi Airport to Phagwara Bus,11:30,09h 15m,20:45,2.9,3300,47 Seats available,https://www.redbus.in/bus-tickets/delhi-airpor...
585,Indo Canadian Transport Pvt. Ltd.,A/C Sleeper (2+1),Delhi Airport to Phagwara Bus,20:00,09h 15m,05:15,2.9,2500,51 Seats available,https://www.redbus.in/bus-tickets/delhi-airpor...
586,Indo Canadian Transport Pvt. Ltd.,A/C Sleeper (2+1),Delhi Airport to Phagwara Bus,22:00,09h 15m,07:15,2.9,3000,42 Seats available,https://www.redbus.in/bus-tickets/delhi-airpor...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Bus_Punjab.csv"
df_buses_Punjab.to_csv(path,index=False)

# 10.Chandigarh

In [ ]:

# Open the browser
driver_10 = webdriver.Chrome()

# Load the webpage
driver_10.get("https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu")

time.sleep(3)  # Wait for the page to load

driver_10.maximize_window()

# Retrieve bus links and routes
wait = WebDriverWait(driver_10, 20)

def Chandigarh_link_route(path):
    Chandigarh_link = []
    Chandigarh_route = []

    # Retrieve the route links
    for i in range(1, 6):
        paths = driver_10.find_elements(By.XPATH, path)

        for link in paths:
            d = link.get_attribute("href")
            Chandigarh_link.append(d)

        # Retrieve names of the routes
        for route in paths:
            Chandigarh_route.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            next_button = pagination.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()={i+1}]')
            time.sleep(3)

            # Retry clicking if intercepted
            retries = 6
            while retries > 0:
                try:
                    next_button.click()
                    break
                except ElementClickInterceptedException:
                    time.sleep(1)
                    retries -= 1
                    if retries == 0:
                        raise

        except NoSuchElementException:
            print(f"Total number of pages found {i}")
            break

    return Chandigarh_link, Chandigarh_route

Chandigarh_link, Chandigarh_route = Chandigarh_link_route("//a[@class='route']")

Total number of pages found 5


In [ ]:
df_C=pd.DataFrame({"Route_name":Chandigarh_route,"Route_link":Chandigarh_link})
df_C

,Route_name,Route_link
0,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...
1,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...
2,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
3,Ludhiana to Chandigarh,https://www.redbus.in/bus-tickets/ludhiana-to-...
4,Chandigarh to Yamuna Nagar,https://www.redbus.in/bus-tickets/chandigarh-t...
5,Chandigarh to Ludhiana,https://www.redbus.in/bus-tickets/chandigarh-t...
6,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
7,Chandigarh to Vrindavan,https://www.redbus.in/bus-tickets/chandigarh-t...
8,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
9,Chandigarh to Pathankot,https://www.redbus.in/bus-tickets/chandigarh-t...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Chandigarh.csv"
df_C.to_csv(path,index=False)

In [ ]:
# Define lists to store the bus details

def  scroll():
    x=0
    l=[]
    while True:
        x=x+1
        driver_10.execute_script('scrollBy(0,100)')
        time.sleep(0.5)
        if(x>1000):
            break

wait = WebDriverWait(driver_10,20)

Chandigarh_bus_data=[]

def scrape_data(Chandigarh_link):

    # Open the route link
    driver_10.get(Chandigarh_link)
    time.sleep(3)
    Busroute=driver_10.find_element(By.XPATH,"//h1[@class='D136_h1']").text
    # Click to view bus details
    try:
        b=driver_10.find_elements(By.CSS_SELECTOR,"div[class='button']")
        for i in range(len(b)-1,-1,-1):
            b[i].click()

        # Scroll to the bottom of the page
        scroll()

        bus_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
        for bus in bus_items:
            try:
                busname = bus.find_element(By.CLASS_NAME, "travels").text
                bustype = bus.find_element(By.CLASS_NAME, "bus-type").text
                departing_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                duration = bus.find_element(By.CLASS_NAME, "dur").text
                reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text
                star_rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                price = bus.find_element(By.CSS_SELECTOR, ".seat-fare .fare span").text
                seats_available = bus.find_element(By.CLASS_NAME, "seat-left").text

                Chandigarh_bus_data.append({
                    "Busname": busname,
                    "Bustype": bustype,
                    "Busroutes": Busroute,
                    "Departing_time": departing_time,
                    "Duration": duration,
                    "Reaching_time": reaching_time,
                    "Star_rating": star_rating,
                    "Price": price,
                    "Seats_available": seats_available,
                    "buslink": Chandigarh_link
                })

            except Exception as e:
                print(f"An error occurred while scraping the data: {e}")
    except Exception as e:
        print(f"An error occurred while trying to click the view buttons: {e}")

for i in Chandigarh_link:
    scrape_data(i)
    time.sleep(3)

print("Data loaded successfully")

An error occurred while scraping the data: Message: no such element: Unable to locate element: {"method":"css selector","selector":".rating-sec .rating span"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF79D1CB645+29573]
	(No symbol) [0x00007FF79D140470]
	(No symbol) [0x00007FF79CFFB6EA]
	(No symbol) [0x00007FF79D04F815]
	(No symbol) [0x00007FF79D04FA6C]
	(No symbol) [0x00007FF79D04312C]
	(No symbol) [0x00007FF79D07733F]
	(No symbol) [0x00007FF79D042FF6]
	(No symbol) [0x00007FF79D077510]
	(No symbol) [0x00007FF79D0986BC]
	(No symbol) [0x00007FF79D0770A3]
	(No symbol) [0x00007FF79D0412DF]
	(No symbol) [0x00007FF79D042441]
	GetHandleVerifier [0x00007FF79D4FC58D+3375821]
	GetHandleVerifier [0x00007FF79D547987+3684039]
	GetHandleVerifier [0x00007FF79D53CDAB+3640043]
	GetHandleVerifier [0x00007FF79D28B7C6+816390]

In [ ]:
df_buses_Chandigarh = pd.DataFrame(Chandigarh_bus_data)
df_buses_Chandigarh

,Busname,Bustype,Busroutes,Departing_time,Duration,Reaching_time,Star_rating,Price,Seats_available,buslink
0,Chandigarh Transport Undertaking (CTU) - 165750,HVAC Seater (2+3),Yamuna Nagar to Chandigarh Bus,12:20,02h 45m,15:05,4.0,195.6,45 Seats available,https://www.redbus.in/bus-tickets/yamuna-nagar...
1,Chandigarh Transport Undertaking (CTU) - 165752,HVAC Seater (2+3),Yamuna Nagar to Chandigarh Bus,14:20,02h 45m,17:05,4.1,195.6,44 Seats available,https://www.redbus.in/bus-tickets/yamuna-nagar...
2,Zimindara Travels,AC Sleeper (2+1),Yamuna Nagar to Chandigarh Bus,20:30,04h 00m,00:30,4.0,849,12 Seats available,https://www.redbus.in/bus-tickets/yamuna-nagar...
3,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),VE A/C Seater / Sleeper (2+1),Yamuna Nagar to Chandigarh Bus,20:40,02h 50m,23:30,3.7,899,21 Seats available,https://www.redbus.in/bus-tickets/yamuna-nagar...
4,City Land Travels,A/C Seater/Sleeper (2+1),Yamuna Nagar to Chandigarh Bus,22:20,02h 00m,00:20,3.5,948,16 Seats available,https://www.redbus.in/bus-tickets/yamuna-nagar...
...,...,...,...,...,...,...,...,...,...,...
908,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),Bharat Benz A/C Seater /Sleeper (2+1),Chandigarh to Haldwani Bus,23:15,10h 30m,09:45,3.7,1199,21 Seats available,https://www.redbus.in/bus-tickets/chandigarh-t...
909,Shree Mahaveer Travels Jaipur,A/C Seater/Sleeper (2+1),Chandigarh to Haldwani Bus,21:00,09h 55m,06:55,3.2,799,22 Seats available,https://www.redbus.in/bus-tickets/chandigarh-t...
910,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),Chandigarh to Firozpur Bus,15:25,04h 05m,19:30,3.6,650,38 Seats available,https://www.redbus.in/bus-tickets/chandigarh-t...
911,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),Chandigarh to Firozpur Bus,17:25,03h 55m,21:20,4.4,650,43 Seats available,https://www.redbus.in/bus-tickets/chandigarh-t...


In [ ]:
# change dataframe to csv
path=r"C:/Users/new/Desktop/Redbus_Project/Bus_Chandigarh.csv"
df_buses_Chandigarh.to_csv(path,index=False)

# Combining 10 States Together

In [ ]:
df_States_1 = pd.read_csv("Kerala.csv")
df_States_2 = pd.read_csv("Andhra.csv")
df_States_3 = pd.read_csv("Telangana.csv")
df_States_4 = pd.read_csv("Rajasthan.csv")
df_States_5 = pd.read_csv("South Bengal.csv")
df_States_6 = pd.read_csv("Himachal.csv")
df_States_7 = pd.read_csv("Assam.csv")
df_States_8 = pd.read_csv("West Bengal.csv")
df_States_9 = pd.read_csv("Punjab.csv")
df_States_10 = pd.read_csv("Chandigarh.csv")

In [ ]:
df_States=pd.concat([df_States_1,df_States_2,df_States_3,df_States_4,df_States_5,df_States_6,df_States_7,df_States_8,df_States_9,df_States_10],ignore_index=True)
df_States

,Route_name,Route_link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Bangalore to Kannur,https://www.redbus.in/bus-tickets/bangalore-to...
...,...,...
328,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...
329,Firozpur to Chandigarh,https://www.redbus.in/bus-tickets/ferozpur-to-...
330,Hansi to Chandigarh,https://www.redbus.in/bus-tickets/hansi-to-cha...
331,Chandigarh to Haldwani,https://www.redbus.in/bus-tickets/chandigarh-t...


In [ ]:
# change dataframe to csv
path=r"C:/Users/New/Desktop/Redbus_Project/States.csv"
df_States.to_csv(path,index=False)

In [ ]:
Redbus_Details_1 = pd.read_csv("Bus_Kerala.csv")
Redbus_Details_2 = pd.read_csv("Bus_Andhra.csv")
Redbus_Details_3 = pd.read_csv("Bus_Telangana.csv")
Redbus_Details_4 = pd.read_csv("Bus_Rajasthan.csv")
Redbus_Details_5 = pd.read_csv("Bus_South Bengal.csv")
Redbus_Details_6 = pd.read_csv("Bus_Himachal.csv")
Redbus_Details_7 = pd.read_csv("Bus_Assam.csv")
Redbus_Details_8 = pd.read_csv("Bus_West Bengal.csv")
Redbus_Details_9 = pd.read_csv("Bus_Punjab.csv")
Redbus_Details_10 = pd.read_csv("Bus_Chandigarh.csv")

In [ ]:
Redbus_Details=pd.concat([Redbus_Details_1,Redbus_Details_2,Redbus_Details_3,Redbus_Details_4,Redbus_Details_5,Redbus_Details_6,Redbus_Details_7,Redbus_Details_8,Redbus_Details_9,Redbus_Details_10],ignore_index=True)
Redbus_Details

,Busname,Bustype,Busroutes,Departing_time,Duration,Reaching_time,Star_rating,Price,Seats_available,buslink
0,KSRTC (Kerala) - 581,Super Express Non AC Seater Air Bus (2+2),Bangalore to Kozhikode Bus,10:00,08h 22m,18:22,4.1,469.0,7 Seats available,https://www.redbus.in/bus-tickets/bangalore-to...
1,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,Bangalore to Kozhikode Bus,12:00,08h 20m,20:20,3.7,627.0,8 Seats available,https://www.redbus.in/bus-tickets/bangalore-to...
2,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,Bangalore to Kozhikode Bus,14:15,08h 50m,23:05,4.3,627.0,32 Seats available,https://www.redbus.in/bus-tickets/bangalore-to...
3,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),Bangalore to Kozhikode Bus,15:00,10h 00m,01:00,3.9,620.0,29 Seats available,https://www.redbus.in/bus-tickets/bangalore-to...
4,KSRTC (Kerala) - 370,AC MULTI AXLE,Bangalore to Kozhikode Bus,15:25,06h 55m,22:20,2.7,729.0,5 Seats available,https://www.redbus.in/bus-tickets/bangalore-to...
...,...,...,...,...,...,...,...,...,...,...
11686,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),Bharat Benz A/C Seater /Sleeper (2+1),Chandigarh to Haldwani Bus,23:15,10h 30m,09:45,3.7,1199.0,21 Seats available,https://www.redbus.in/bus-tickets/chandigarh-t...
11687,Shree Mahaveer Travels Jaipur,A/C Seater/Sleeper (2+1),Chandigarh to Haldwani Bus,21:00,09h 55m,06:55,3.2,799.0,22 Seats available,https://www.redbus.in/bus-tickets/chandigarh-t...
11688,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),Chandigarh to Firozpur Bus,15:25,04h 05m,19:30,3.6,650.0,38 Seats available,https://www.redbus.in/bus-tickets/chandigarh-t...
11689,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),Chandigarh to Firozpur Bus,17:25,03h 55m,21:20,4.4,650.0,43 Seats available,https://www.redbus.in/bus-tickets/chandigarh-t...


In [ ]:
# change dataframe to csv
path=r"C:/Users/New/Desktop/Redbus_Project/Redbus Details.csv"
Redbus_Details.to_csv(path,index=False)

In [ ]:
Redbus_Details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11691 entries, 0 to 11690
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Busname          11691 non-null  object 
 1   Bustype          11691 non-null  object 
 2   Busroutes        11691 non-null  object 
 3   Departing_time   11691 non-null  object 
 4   Duration         11691 non-null  object 
 5   Reaching_time    11691 non-null  object 
 6   Star_rating      11691 non-null  float64
 7   Price            11691 non-null  float64
 8   Seats_available  11691 non-null  object 
 9   buslink          11691 non-null  object 
dtypes: float64(2), object(8)
memory usage: 913.5+ KB


In [ ]:
Route_name= pd.read_csv("Redbus Details.csv")
Busname_1 = Route_name.iloc[:,2]
# Get unique values and convert to list
uniquevalue_1 = Busname_1.unique().tolist()
unique_df = pd.DataFrame(uniquevalue_1, columns=['Busroutes'])
# Save the new DataFrame to a CSV file
unique_df.to_csv('unique_bus_routes.csv', index=False)

# Creating Mysql database

In [ ]:
# importing libraries
import pymysql

In [ ]:
con= pymysql.connect(
    host="localhost",
    user="root",
    password="Mirthu@243",
    autocommit=True
    )
mycursor=con.cursor()
mycursor.execute("CREATE DATABASE REDBUS_DETAILS")

1

In [ ]:
mycursor.execute("USE REDBUS_DETAILS")

0

In [ ]:
# Creating Table
mycursor.execute('''CREATE TABLE bus_routes(
                  ID INT AUTO_INCREMENT PRIMARY KEY,
                  Busname VARCHAR(255) NOT NULL,
                  Bustype VARCHAR(255) NOT NULL,
                  Busroutes VARCHAR(255) NULL,
                  Departing_time VARCHAR(255) NOT NULL,
                  Duration VARCHAR(255) NOT NULL,
                  Reaching_time VARCHAR(255) NOT NULL,
                  Star_rating Float NULL,
                  Price FLOAT NULL,
                  Seats_available VARCHAR(255) NOT NULL,
                  buslink VARCHAR(255) NULL
                  )''')
print("Table Created successfully")

Table Created successfully


In [ ]:
# Inserting data into bus_routes table
insert_query = '''INSERT INTO bus_routes(
                    Busname,
                    Bustype,
                    Busroutes,
                    Departing_time,
                    Duration,
                    Reaching_time,
                    Star_rating,
                    Price,
                    Seats_available,
                    buslink
                    )
                  VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
data = Redbus_Details.values.tolist()

mycursor.executemany(insert_query, data)

con.commit()

print("Values inserted successfully")

Values inserted successfully
